In [1]:
# TODO: fix last dense layer (should be 10 but only accespts 8). Is it the number of classes? If so then nothing to fix
# TODO: discuss how reading in dataset could be made multi-threaded (each dataset is read in on own thread then all joined together at the end)

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import imageio as iio
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from PIL import Image
from numba import cuda  # https://stackoverflow.com/a/52354865/6476994
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv
import re

In [2]:
# allows all images to be displayed at once (else only displays the last call to plt.imshow())
# https://stackoverflow.com/a/41210974
def displayImage(image, caption = None, colour = None) -> None:
    plt.figure()
    if(colour != None):
        plt.imshow(image, cmap=colour)
    else:
        plt.imshow(image)
        
    if(caption != None):
        # display caption below picture (https://stackoverflow.com/a/51486361)
        plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

In [3]:
dataset_names = ["BB01", "BB02", "BB03", "BB04", "BB05", "BB06", "BB07", "BB08", "BB09", "BB10"]
# dataset_names = ["BB08"]
# dataset_names = []
for i in range(11, 21):
    dataset_names.append("BB{}".format(i))
print("dataset_names: {}".format(dataset_names))

dataset_names: ['BB01', 'BB02', 'BB03', 'BB04', 'BB05', 'BB06', 'BB07', 'BB08', 'BB09', 'BB10', 'BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 'BB19', 'BB20']


In [4]:
# free up GPU if it didn't after the last run
cuda.select_device(0)
cuda.close()

In [5]:
# mnist = tf.keras.datasets.mnist
# (training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# Read in dataset

In [6]:
# get the all original output filenames
def readInImages(datasetName):
    print("reading in images for dataset: {}".format(datasetName))
    desired_size = 224
    image_list = []
    imgRegExp = re.compile(r'.*[.](JPG)$')
    # https://stackoverflow.com/a/3207973
    all_image_filenames = next(os.walk('data/{}'.format(datasetName)),
                         (None, None, []))[2]  # [] if no file
    # filter out file names that are not JPEGs
    all_image_filenames = [i for i in all_image_filenames if imgRegExp.match(i)]
    # walk() outputs unordered, so we need to sort
    all_image_filenames.sort()
    # print("all_image_filenames: {}".format(all_image_filenames))
    print("all_image_filenames length: {}".format(len(all_image_filenames)))
    for fn in all_image_filenames:
        # im = Image.open('data/{}/{}'.format(datasetName, fn))
        im = cv2.imread('data/{}/{}'.format(datasetName, fn))
        # resize the image to conserve memory, and transform it to be square while
        # maintaining the aspect ration (give it padding):
        # https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/#using-opencv
        # im = cv2.resize(im, (480, 270), interpolation=cv2.INTER_CUBIC)
        # im = cv2.resize(im, (240, 135), interpolation=cv2.INTER_CUBIC)
        # im = cv2.resize(im, (192, 108), interpolation=cv2.INTER_CUBIC)
        old_size = im.shape[:2]
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        im = cv2.resize(im, (new_size[1], new_size[0]))

        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)

        color = [0, 0, 0]
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

        
        image_list.append(np.asarray(new_im))
    
    print("done current dataset")
    return image_list

all_images = []
for fn in dataset_names:    
    all_images = [*all_images, *readInImages(fn)]

reading in images for dataset: BB01
all_image_filenames length: 285
done current dataset
reading in images for dataset: BB02
all_image_filenames length: 45
done current dataset
reading in images for dataset: BB03
all_image_filenames length: 230
done current dataset
reading in images for dataset: BB04
all_image_filenames length: 999
done current dataset
reading in images for dataset: BB05
all_image_filenames length: 189
done current dataset
reading in images for dataset: BB06
all_image_filenames length: 1137
done current dataset
reading in images for dataset: BB07
all_image_filenames length: 324
done current dataset
reading in images for dataset: BB08
all_image_filenames length: 66
done current dataset
reading in images for dataset: BB09
all_image_filenames length: 357
done current dataset
reading in images for dataset: BB10
all_image_filenames length: 121
done current dataset
reading in images for dataset: BB11
all_image_filenames length: 167
done current dataset
reading in images for 

# Read in dataset's labels

In [7]:
# labels (using dataset's CSV file)

def readInAnnotations(datasetName):
    labelList = []
    # https://realpython.com/python-csv/#reading-csv-files-with-csv
    with open('data/{}/{}.csv'.format(datasetName, datasetName)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            # print("row: {}".format(row))
            # first row always contains this string, so ignore it
            if "RECONYX - MapView Professional" in row:
                continue
            if line_count == 0:
                # print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                # print("Image Name: {}. Hit List: {}".format(row[0], row[22].replace("\n", ", ")))
                # FIXME handle when hitlist contains more than one item (e.g., BB06 IMG_512 has 'kangaroo' and 'empty photo') - sort of handled, need to make more dynamic
                hit_list = row[22]
                if hit_list == '':
                    labelList.append("Empty photo")
                elif hit_list == 'Empty photo\nHuman Presense/Deployment':
                    labelList.append("Human Presense/Deployment")
                elif hit_list == 'Kangaroo\nEmpty photo':
                    labelList.append("Kangaroo")
                else:
                    # FIXME: rendundant case?
                    labelList.append(hit_list.replace("\n", ", "))
                line_count += 1
    # print("returning labelList (length: {}): {}".format(len(labelList), labelList))
    # print("returning labelList of length: {}".format(len(labelList)))
    return labelList

all_image_labels = []
for fn in dataset_names:
    all_image_labels = [*all_image_labels, *readInAnnotations(fn)]

# print("all_image_labels: {}".format(all_image_labels))

classes = set(all_image_labels)
print("all classes (length={}): {}".format(len(classes), classes))

all classes (length=8): {'Human Presense/Deployment', 'Other', 'Cat', 'Fox', 'Emu', 'Kangaroo', 'Rabbit', 'Empty photo'}


# Randomly split the dataset and corresponding labels into training and test sets

In [8]:
print("all_images size: {}".format(len(all_images)))
# print("all_image_labels size: {}".format(len(all_image_labels)))


training_images, test_images, training_labels, test_labels = train_test_split(all_images, all_image_labels, test_size=0.2, random_state=42)

print("training_labels length: {}".format(len(training_labels)))
print("test_labels length: {}".format(len(test_labels)))

training_classes = set(training_labels)
test_classes = set(test_labels)
print("training_classes (length={}): {}".format(len(training_classes), training_classes))
print("test_classes (length={}): {}".format(len(test_classes), test_classes))

# integer-encode labels so they can be one-hot-encoded
# https://stackoverflow.com/a/56227965/6476994
label_encoder = LabelEncoder()
training_labels = np.array(training_labels)
training_labels = label_encoder.fit_transform(training_labels)
test_labels = np.array(test_labels)
test_labels = label_encoder.fit_transform(test_labels)


# convert list of numpy arrays to numpy array of numpy arrays
# https://stackoverflow.com/a/27516930/6476994
training_images = np.stack(training_images, axis = 0)
test_images = np.stack(test_images, axis = 0)

print("done stacking")
print("training_images shape: {}".format(training_images.shape))
print("test_images shape: {}".format(test_images.shape))

all_images size: 5067
training_labels length: 4053
test_labels length: 1014
training_classes (length=8): {'Human Presense/Deployment', 'Other', 'Cat', 'Fox', 'Emu', 'Kangaroo', 'Rabbit', 'Empty photo'}
test_classes (length=8): {'Human Presense/Deployment', 'Other', 'Cat', 'Fox', 'Emu', 'Kangaroo', 'Rabbit', 'Empty photo'}
done stacking
training_images shape: (4053, 224, 224, 3)
test_images shape: (1014, 224, 224, 3)


# ZFNet

Source: https://towardsdatascience.com/zfnet-an-explanation-of-paper-with-code-f1bd6752121d

## Train the model

In [9]:
# training_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), training_images).numpy()
# test_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), test_images).numpy()

training_images = tf.image.resize(training_images, [224, 224]).numpy()
test_images = tf.image.resize(test_images, [224, 224]).numpy()

training_images = training_images.reshape(training_images.shape)
training_images = training_images / 255.0
test_images = test_images.reshape(test_images.shape)
test_images = test_images / 255.0

training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=len(training_classes))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(test_classes))

num_len_train = int(0.8 * len(training_images))

ttraining_images = training_images[:num_len_train]
ttraining_labels = training_labels[:num_len_train]

valid_images = training_images[num_len_train:]
valid_labels = training_labels[num_len_train:]

training_images = ttraining_images
training_labels = ttraining_labels

model = tf.keras.models.Sequential([
                                    
		tf.keras.layers.Conv2D(96, (7, 7), strides=(2, 2), activation='relu',
			input_shape=(224, 224, 3)),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), activation='relu'),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),

		tf.keras.layers.MaxPooling2D(3, strides=2),

    tf.keras.layers.Flatten(),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(len(classes), activation='softmax')#FIXME is this the number of classes? (check paper)
	])


model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9), \
              loss='categorical_crossentropy', \
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', \
                                            		factor=0.1, patience=1, \
																								min_lr=0.00001)

model.fit(training_images, training_labels, batch_size=128, \
          validation_data=(valid_images, valid_labels), \
					epochs=90, callbacks=[reduce_lr])

2022-05-28 09:34:08.470131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 09:34:08.473566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 09:34:08.473801: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 09:34:08.474619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/90


2022-05-28 09:34:13.236682: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-05-28 09:34:13.563447: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


26/26 [==============================] - 9s 214ms/step - loss: 1.5686 - accuracy: 0.5608 - top_k_categorical_accuracy: 0.9565 - val_loss: 1.3435 - val_accuracy: 0.6412 - val_top_k_categorical_accuracy: 0.9840 - lr: 0.0100
Epoch 2/90
26/26 [==============================] - 4s 162ms/step - loss: 1.0667 - accuracy: 0.6410 - top_k_categorical_accuracy: 0.9861 - val_loss: 1.0795 - val_accuracy: 0.6449 - val_top_k_categorical_accuracy: 0.9901 - lr: 0.0100
Epoch 3/90
26/26 [==============================] - 4s 162ms/step - loss: 0.9892 - accuracy: 0.6481 - top_k_categorical_accuracy: 0.9907 - val_loss: 0.9682 - val_accuracy: 0.6671 - val_top_k_categorical_accuracy: 0.9889 - lr: 0.0100
Epoch 4/90
26/26 [==============================] - 4s 163ms/step - loss: 0.9136 - accuracy: 0.6672 - top_k_categorical_accuracy: 0.9920 - val_loss: 0.9017 - val_accuracy: 0.6930 - val_top_k_categorical_accuracy: 0.9889 - lr: 0.0100
Epoch 5/90
26/26 [==============================] - 4s 163ms/step - loss: 0.863

## Evaluate the trained model

In [10]:
print('test_images shape: {}'.format(test_images.shape))
print('test_labels shape: {}'.format(test_labels.shape))

results = model.evaluate(test_images,test_labels)

test_images shape: (1014, 224, 224, 3)
test_labels shape: (1014, 8)
32/32 [==============================] - 1s 21ms/step - loss: 0.5641 - accuracy: 0.8245 - top_k_categorical_accuracy: 0.9931


In [11]:
predictions = (model.predict(test_images) > 0.5).astype("int32")
print("Predictions (shape: {}):\n{}".format(predictions.shape, predictions))

32/32 [==============================] - 1s 14ms/step
Predictions (shape: (1014, 8)):
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [12]:
print(classification_report(test_labels, predictions))
# TODO resolve classes to their names (current integers in left column)

# from sklearn.metrics import precision_score
# print("Precision score: {}".format(precision_score(test_labels,predictions)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.77      0.54      0.64       160
           2       0.87      0.71      0.78        97
           3       0.62      0.29      0.39        28
           4       0.79      0.39      0.52        59
           5       0.89      0.94      0.92       644
           6       0.00      0.00      0.00        14
           7       0.00      0.00      0.00         3

   micro avg       0.87      0.78      0.82      1014
   macro avg       0.49      0.36      0.41      1014
weighted avg       0.83      0.78      0.80      1014
 samples avg       0.78      0.78      0.78      1014



/home/luke/miniconda3/envs/cv-project-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luke/miniconda3/envs/cv-project-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
model.save('ZFNet.h5')

## Free up the GPU's memory

In [14]:
cuda.select_device(0)
cuda.close()

In [15]:
# from collections import Counter
# Counter(training_labels.tolist())
# Counter(test_labels.tolist())